# Transhipment: optimality conditions

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from itertools import chain
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from networkx import (
    DiGraph,
    Graph,
    draw_networkx_nodes,
    draw_networkx_labels,
    draw_networkx_edges,
    draw_networkx_edge_labels,
)


In this lab, you will study **optimality conditions** for a transhipment model by working with its
dual variables and inequalities. You will formulate the dual of the standard-form network model,
update node potentials using edge costs to enforce the dual inequalities, and verify optimality on
each arc. Then, using **complementarity slackness**, you will recover a primal flow and interpret it
on the network. The goal is to connect the algebra (Lagrangian, dual constraints), the geometry
(feasible flows on arcs and cuts), and the theory (weak/strong duality).

A node in the_network can be of any type. In this script, we use str.

In [ ]:
Node = Any


Consider three factories producing electric cars in Boston, New-York
and Los Angeles, with supplies 200 units, 250 units and 300 units per
day respectively. The demand is 250 cars per day and per city. The
cost for transporting one car:

- from Los Angeles to Boston is \$100.
- from Los Angeles to New-York is \$50.
- from New-York to Boston is \$80.

No more than 40 cars can be transported between two cities per day.

Create a directed graph

In [ ]:
original_network = DiGraph()


Supply and demand at each factory

In [ ]:
original_network.add_node('LA', supply=300 - 250)
original_network.add_node('NY', supply=250 - 250)
original_network.add_node('B', supply=200 - 250)


Add arcs with cost, and capacity (lower and upper bound)

In [ ]:
original_network.add_edge('LA', 'B', cost=100, lower_bound=0, upper_bound=40)
original_network.add_edge('LA', 'NY', cost=50, lower_bound=0, upper_bound=40)
original_network.add_edge('NY', 'B', cost=80, lower_bound=0, upper_bound=40)


Define coordinates for the plot.

In [ ]:
pos = {'LA': (0, 3), 'NY': (-3, 0), 'B': (3, 0)}



Function to plot the the_network

In [ ]:
def plot_network(network: Graph, positions: dict[Node, tuple[float, float]]) -> None:
    """Plot the the_network and its data.

    :param network: the_network to plot.
    :param positions: coordinate sof the nodes
    """

    # Figure size
    plt.figure(figsize=(10, 10))

    # Draw the nodes
    draw_networkx_nodes(
        network, positions, node_size=5000, node_color='lightblue', alpha=0.5
    )

    # Draw the node labels (supply/demand)
    node_labels = {}
    for node, data in network.nodes(data=True):
        supply = data['supply']
        node_labels[node] = f'{node} [{supply}]'

    shifted_positions = {
        node: (coord[0], coord[1] - 0.2) for node, coord in positions.items()
    }
    draw_networkx_labels(
        network, shifted_positions, labels=node_labels, font_size=12, font_weight='bold'
    )

    # Draw the arcs with labels
    edge_labels = {}
    for u, v, data in network.edges(data=True):
        lower_bound = data['lower_bound']
        upper_bound = data['upper_bound']
        cost = data['cost']
        label = f'{cost} ({lower_bound},{upper_bound})'
        edge_labels[(u, v)] = label

    draw_networkx_edges(network, positions, arrowstyle='->', arrowsize=20)
    draw_networkx_edge_labels(network, positions, edge_labels=edge_labels, font_size=10)

    # Display the graph
    plt.title('Transhipment the_network with costs and capacities')
    plt.axis('off')
    plt.show()


plot_network(network=original_network, positions=pos)


It corresponds to the following transhipment problem:
$$\min_{x \in \R^3} 100 x_{LA,B} + 50 x_{LA,NY} + 80
x_{NY,B}$$ subject to
\begin{align*}
x_{LA,NY}+x_{LA,B}&=50, \\ x_{NY,B} - x_{LA,NY}&=0,
\\ -x_{NY,B}-x_{LA,B}&=-50, \\ 0 \leq x_{LA,NY},x_{LA,B},x_{NY,B}
&\leq 40.
\end{align*}

In order to obtain a formulation in standard form, we apply the procedure used in the previous exercise.

Shifted the_network

In [ ]:
shifted_network = original_network.copy()

for node, node_data in shifted_network.nodes(data=True):
    correction = 0
    for u, v, edge_data in shifted_network.in_edges(node, data=True):
        correction += edge_data['lower_bound']
    for u, v, edge_data in shifted_network.out_edges(node, data=True):
        correction -= edge_data['lower_bound']
    node_data['supply'] += correction

for u, v, data in shifted_network.edges(data=True):
    lower_bound = data['lower_bound']
    upper_bound = data['upper_bound']
    data['lower_bound'] = 0
    data['upper_bound'] = upper_bound - lower_bound


Standard form the_network.

In [ ]:
standard_form_network = DiGraph()
new_pos = {}
for node, node_data in shifted_network.nodes(data=True):
    # For each node of the shifted the_network, we create a  node in the new the_network, with a different supply data.
    supply = node_data['supply']
    # For each outgoing arc, we deduce the upper bound.
    for u, v, edge_data in shifted_network.out_edges(node, data=True):
        supply -= edge_data['upper_bound']

    standard_form_network.add_node(node, supply=supply)
    new_pos[node] = pos[node]

for u, v, edge_data in shifted_network.edges(data=True):
    # For each arc of the shifted the_network, we create a node in the new the_network, and two arcs: one corresponding to the
    # original arc, and one corresponding to the slack variable.

    # We define the name of the new node.
    slack_node = f'slack_{u}_{v}'
    # The supply of the new node is the upper bound of the corresponding arc.
    supply_new_node = edge_data['upper_bound']
    standard_form_network.add_node(slack_node, supply=supply_new_node)

    # We add an arc between the new node and the downstream node

    # The lower bound is 0
    lower_bound_down_arc = 0
    # The upper bound is not needed anymore. We set it to $\infty$.
    upper_bound_down_arc = np.inf
    # The cost of the new arc is the cost of the corresponding arc, as it plays the role of the original arc.
    cost_down_arc = edge_data['cost']
    standard_form_network.add_edge(
        slack_node,
        v,
        cost=cost_down_arc,
        lower_bound=lower_bound_down_arc,
        upper_bound=upper_bound_down_arc,
    )

    # We also add an arc between the new node and the upstream node

    # The lower bound is 0
    lower_bound_up_arc = 0
    # The upper bound is not needed anymore. We set it to $\infty$.
    upper_bound_up_arc = np.inf
    # The cost of the new arc is 0, as it corresponds to the slack variable.
    cost_up_arc = 0
    standard_form_network.add_edge(
        slack_node,
        u,
        cost=cost_up_arc,
        lower_bound=lower_bound_up_arc,
        upper_bound=upper_bound_up_arc,
    )
    # We position the new node in the middle of the corresponding arc.
    coord_u_x, coord_u_y = pos[u]
    coord_v_x, coord_v_y = pos[v]
    new_pos[slack_node] = (0.5 * (coord_u_x + coord_v_x), 0.5 * (coord_u_y + coord_v_y))


We plot the new the_network.

In [ ]:
plot_network(network=standard_form_network, positions=new_pos)



The corresponding linear optimization problem in standard form
is: $$\min_{x\in{\mathbb{R}^6}} 100 x_{2,B} + 80 x_{3,B} + 50 x_{1,NY}$$ subject to
\begin{align*}
-x_{1,LA}-x_{2,LA} &= -30,\\ x_{1,LA}+x_{1,NY} &=
40,\\ -x_{1,NY}-x_{3,NY} &= -40,\\ x_{3,NY}+x_{3,B} &=
40,\\ -x_{2,B}-x_{3,B} &= -50,\\ x_{2,LA}+x_{2,B} &=
40,\\ x_{1,LA},x_{1,NY},x_{2,LA},x_{2,B},x_{3,NY},x_{3,B} & \geq 0.
\end{align*}

For the readability of the indices, we have used:
- index 1 for slack_LA_NY
- index 2 for slack_LA_B
- index 3 for slack_NY_B

# Question 1
Write the dual problem using the Lagrangian method.

A dual variable $\lambda_i$ is associated with each node of the
transformed the_network, and a dual variable $\mu_{ij}$ is associated
with each arc of the transformed the_network.  The Lagrangian is

$$L(x,\lambda,\mu) = \sum_{(i,j) \in \mathcal{A}} c_{ij} x_{ij} +
\sum_i \lambda_i (\sum_{(i,j) \in \mathcal{A}} x_{ij} - \sum_{(k,i)
\in \mathcal{A}} x_{ki} -s_i) - \sum_{(i,j) \in \mathcal{A}}
\mu_{ij} x_{ij},$$ that is

\begin{align*}
L(x,\lambda,\mu) = & 100 x_{2,B} + 50 x_{1,NY} + 80 x_{3,B} \\ &+
\lambda_\text{LA}(-x_{1,LA}-x_{2,LA}+30) + \lambda_1(x_{1,NY} +
x_{1,LA}-40)\\ &+ \lambda_\text{NY}(-x_{3,NY} - x_{1,NY}+40)
+\lambda_2 (x_{2,LA} + x_{2,B}-40)\\ &+
\lambda_{B}(-x_{2,B}-x_{3,B}+50) +\lambda_3 (x_{3,NY} +
x_{3,B}-40) \\ &- \mu_{1,NY}x_{1,NY}- \mu_{1,LA}x_{1,LA}-
\mu_{2,LA}x_{2,LA}\\ &- \mu_{2,B}x_{2,B}- \mu_{3,B}x_{3,B}-
\mu_{3,NY}x_{3,NY}\\ = &
x_{1,LA}(-\lambda_\text{LA}+\lambda_1-\mu_{1,LA}) +
x_{1,NY}(50-\lambda_\text{NY}+\lambda_1-\mu_{1,NY}) \\ &+
x_{2,LA}(-\lambda_\text{LA}+\lambda_2-\mu_{2,LA}) +
x_{2,B}(100-\lambda_B + \lambda_2-\mu_{2,B})\\ &+
x_{3,NY}(-\lambda_\text{NY}+\lambda_3-\mu_{3,NY})+ x_{3,B} (80 -
\lambda_B +\lambda_3 -
\mu_{3,B})\\ &+30\lambda_\text{LA}+40\lambda_\text{NY}+50\lambda_B-40\lambda_1-40\lambda_2-40\lambda_3.
\end{align*}

The coefficient of each flow variable must be zero to get a bounded
dual problem:
\begin{align*}
-\lambda_\text{LA}+\lambda_1 &= \mu_{1,LA},
\\ 50-\lambda_\text{NY}+\lambda_1 &=\mu_{1,NY},
\\ -\lambda_\text{LA}+\lambda_2 &=\mu_{2,LA}, \\ 100-\lambda_B +
\lambda_2 &=\mu_{2,B}, \\ -\lambda_\text{NY}+\lambda_3 &=
\mu_{3,NY}, \\ 80 - \lambda_B +\lambda_3 &= \mu_{3,B}. \\
\end{align*}

Moreover since $\mu_{ij} \geq 0, \quad \forall i,j$, we can
eliminate the $\mu$'s, and we obtain the following set of
constraints:

\begin{align*}
-\lambda_\text{LA}+\lambda_1 & \geq
0,\\ \lambda_\text{NY}-\lambda_1 & \leq
50,\\ -\lambda_\text{LA}+\lambda_2 & \geq 0,\\ \lambda_B -
\lambda_2 & \leq 100,\\ -\lambda_\text{NY}+\lambda_3 & \geq
0,\\ \lambda_B - \lambda_3 & \leq 80.
\end{align*}

The objective function is:
$$q(\lambda)=30\lambda_\text{LA}+40\lambda_\text{NY}+50\lambda_B-40\lambda_1-40\lambda_2-40\lambda_3.$$

The dual problem is therefore:
$$\max
30\lambda_\text{LA}+40\lambda_\text{NY}+50\lambda_B-40\lambda_1-40\lambda_2-40\lambda_3$$
subject to
\begin{align*}
-\lambda_\text{LA}+\lambda_1 & \geq
0,\\ \lambda_\text{NY}-\lambda_1 & \leq
50,\\ -\lambda_\text{LA}+\lambda_2 & \geq 0,\\ \lambda_B -
\lambda_2 & \leq 100,\\ -\lambda_\text{NY}+\lambda_3 & \geq
0,\\ \lambda_B - \lambda_3 & \leq 80.
\end{align*}

# Question 2
Solve the dual using optimality conditions.  The procedure
consists in treating all nodes of the the_network in a systematic
way. The set of nodes that must be treated at a given iteration is
denoted by $\mathcal{S}$. The treatment of one node consists in
considering all arcs incident to the node, and updating the value of
the dual variable of the other incident node using the optimality
conditions.

- Select one arbitrary node $i$, set $\lambda_i=0$ and
$\mathcal{S}=\{i\}$.
- If $\mathcal{S} \neq \emptyset$, perform
the following:

1. Select a node $i$ in $\mathcal{S}$.
2. For each outgoing arc $(i, j)$, if $\lambda_j$ is not
defined yet, or if it violates the optimality conditions, update
its value:
$$
\text{If } \lambda_j > \lambda_i + c_{ij}, \text{ then }
\lambda_j=\lambda_i + c_{ij} \text { and } \mathcal{S} =
\mathcal{S} \cup \{ j\}.
$$
3. For each ingoing arc $(j, i)$, if $\lambda_j$ is not defined
yet, or if it violates the optimality conditions, update its
value:
$$
\text{If } \lambda_j < \lambda_i - c_{ji}, \text{ then }
\lambda_j=\lambda_i - c_{ji} \text { and } \mathcal{S} =
\mathcal{S} \cup \{ j\}.
$$

4. Remove $i$ from $\mathcal{S}$ and start again.

- If $\mathcal{S} = \emptyset$, stop.

We suggest to initialize the process with node slack_LA_NY, and then process
each node in a counter-clockwise way, that is: slack_LA_NY, NY, slack_NY_B, B, slack_LA_B, LA.

We initialize the dual variables

In [ ]:
dual_variables: dict[Node, float | None] = {
    node: None for node in standard_form_network.nodes()
}

We initialize the set of nodes to be treated

In [ ]:
set_of_nodes_s = set()



We define the procedure that treats a node

In [ ]:
def treat_node(node_to_be_treated: Node) -> None:
    """Apply the procedure to calculate the dual variables
    :param node_to_be_treated: self explanatory...
    """
    print(f'*** Treating node {node_to_be_treated} ***')
    # We treat each arc leaving the node and apply the optimality conditions
    for u, v, edge_data in standard_form_network.out_edges(
        node_to_be_treated, data=True
    ):
        if dual_variables[v] is None:
            # No value has been set yet. We initialize it with the optimality condition
            print(f'Dual variable of {v} is not defined.')
            dual_variables[v] = (
                dual_variables[u] + edge_data['cost']
            )
            # We add the node to the set in order to treat it later.
            set_of_nodes_s.add(v)
        elif (
            dual_variables[u] + edge_data['cost'] < dual_variables[v]
        ):
            # The optimality condition is violated. We update the dual variable.
            print(f'Optimality condition violated for edge {(u, v)}')
            dual_variables[v] = (
                dual_variables[u] + edge_data['cost']
            )
            # We add the node to the set in order to treat it later.
            set_of_nodes_s.add(v)
        else:
            # The optimality condition is verified. There is nothing to do.
            print(f'Optimality condition verified for edge {(u, v)}')

    # We treat each arc entering the node and apply the optimality conditions.
    # Keep in mind that the role of u and v change here.
    for u, v, edge_data in standard_form_network.in_edges(
        node_to_be_treated, data=True
    ):
        if dual_variables[u] is None:
            # No value has been set yet. We initialize it with the optimality condition
            print(f'Dual variable of {u} is not defined.')
            dual_variables[u] = (
                dual_variables[v] - edge_data['cost']
            )
            # We add the node to the set in order to treat it later.
            set_of_nodes_s.add(u)
        elif (
            dual_variables[u] + edge_data['cost'] < dual_variables[v]
        ):
            # The optimality condition is violated. We update the dual variable.
            print(f'Optimality condition violated for edge {(u, v)}')
            dual_variables[u] = (
                dual_variables[v] - edge_data['cost']
            )
            # We add the node to the set in order to treat it later.
            set_of_nodes_s.add(u)
        else:
            # The optimality condition is verified. There is nothing to do.
            print(f'Optimality condition verified for edge {(u, v)}')
    set_of_nodes_s.remove(node_to_be_treated)



We now apply the procedure node by node.

We start with node slack_LA_NY

In [ ]:
treated_node = 'slack_LA_NY'
set_of_nodes_s.add(treated_node)
dual_variables[treated_node] = 0


Dual variables

In [ ]:
print(dual_variables)


Set of nodes

In [ ]:
print(set_of_nodes_s)


We treat the node

In [ ]:
treat_node(node_to_be_treated=treated_node)


Dual variables

In [ ]:
print(dual_variables)


Set of nodes

In [ ]:
print(set_of_nodes_s)


We now treat the node 'NY', which is in the set.

In [ ]:
treat_node(node_to_be_treated='NY')


Dual variables

In [ ]:
print(dual_variables)


Set of nodes

In [ ]:
print(set_of_nodes_s)


We now treat the node 'slack_NY_B', which is in the set.

In [ ]:
treat_node(node_to_be_treated='slack_NY_B')


Dual variables

In [ ]:
print(dual_variables)


Set of nodes

In [ ]:
print(set_of_nodes_s)


We now treat the node 'B', which is in the set.

In [ ]:
treat_node(node_to_be_treated='B')


Dual variables

In [ ]:
print(dual_variables)


Set of nodes

In [ ]:
print(set_of_nodes_s)


We now treat the node 'slack_LA_B', which is in the set.

In [ ]:
treat_node(node_to_be_treated='slack_LA_B')


Dual variables

In [ ]:
print(dual_variables)


Set of nodes

In [ ]:
print(set_of_nodes_s)


We now treat the node 'LA', which is in the set.

In [ ]:
treat_node(node_to_be_treated='LA')


Dual variables

In [ ]:
print(dual_variables)


Set of nodes

In [ ]:
print(set_of_nodes_s)



The set is now empty. We have found the optimal values for the dual variables.
Indeed, the optimality conditions are verified on each arc.

In [ ]:
def verify_optimality(the_dual_variables: dict[Node, float]) -> bool:
    """Check the optimality of the dual variables

    :param the_dual_variables: dict of value for the dual variables.
    :return: True if verified. False otherwise.
    """
    for u, v, edge_data in standard_form_network.edges(data=True):
        optimality_condition = (
            edge_data['cost'] + the_dual_variables[u] - the_dual_variables[v]
        )
        print(
            f'Arc {(u,v)}: {optimality_condition}. Verified? {optimality_condition >= 0}'
        )
    return optimality_condition >= 0



We check the optimality of the dual variables.

In [ ]:
check = verify_optimality(the_dual_variables=dual_variables)
print(f'Are all optimality conditions verified? {check}')


## Question 3

Consider another solution for the dual variables.

In [ ]:
other_dual_variables = {
    'LA': 100,
    'NY': 150,
    'B': 230,
    'slack_LA_B': 130,
    'slack_LA_NY': 100,
    'slack_NY_B': 150,
}


Check if it is an optimal solution.

In [ ]:
check = verify_optimality(the_dual_variables=other_dual_variables)
print(f'Are all optimality conditions verified? {check}')


It is also an optimal solution. Actually, in the optimality conditions, only the difference of the dual
variables matter. Therefore, if you add a constant to each variable (here, 100), you obtain another
optimal solution.

## Question 4

Use the complementarity slackness theorem and the flow conservation constraints to find the optimal flow, that is,
the optimal values of the primal variables.

For the readability of the indices, we use again:
- index 1 for slack_LA_NY
- index 2 for slack_LA_B
- index 3 for slack_NY_B

We note that the arc (slack_LA_B, LA) is the only one where the
inequality is strict. Therefore, we now that this arc carries no
flow:
$$
x^*_{2,\text{LA}} = 0.
$$

Therefore, in order to verify the flow conservation constraints at
node LA, the demand must be served by the arc $(1,\text{LA})$, so
that
$$
x^*_{1,\text{LA}} = 30.
$$
At node 1, the flow conservation constraints are verified if
$$
x^*_{1,\text{NY}} = 10.
$$
Applying the flow conservation constraints are node NY, we obtain
$$
x^*_{3,\text{NY}} = 30.
$$
At node 3, we obtain
$$
x^*_{3,B} = 10.
$$
At node B, we obtain
$$
x^*_{2,B} = 40.
$$
And we can check that the flow conservation constraints are verified
at node 2 (slack_LA_B). If we now ignore the slack variables, and come back to
the original formulation, this can be interpreted as:

- sending 10 units of flow from LA to NY, for a total cost of 500,
- sending 40 units of flow from LA to B, for a total cost of 4000,
- sending 10 units of flow from NY to B, for a total cost of 800.

The total cost is therefore 5300. Note that the objective function
of the dual is
\begin{align*}
q(\lambda)&=30\lambda_\text{LA}+40\lambda_\text{NY}+50\lambda_B-40\lambda_1-40\lambda_2-40\lambda_3,
\\ &= 0 +2000+6500+0-1200-2000 \\ &= 5300,
\end{align*}
consistently with the strong duality theorem.